# **Project: E-commerce AI Assistan**t

This notebook is a development environment for building and testing the core AI functionalities of the E-commerce AI Assistant.

Key Features:
-   **Analytics Chatbot:** A powerful tool that allows users to upload a CSV file of their e-commerce data. Users can ask natural language questions about their sales, customer behavior, and product performance. The chatbot then provides data-driven insights and strategic recommendations.
-   **RAG Assistant:** A dedicated knowledge base assistant. This application uses Retrieval-Augmented Generation (RAG) to search through internal documentation or product information and provide quick, contextual, and accurate answers to specific queries.

This is a working document for prototyping and model evaluation.


In [ ]:
import warnings
warnings.filterwarnings("ignore")

# *Ecommerce Analytics Chatbot*
This is our Main Featyre in The system
## Environment Setup & Library Installation
Here we'll prepare all the necessary tools and libraries to build our application.

In [ ]:
# Install required libraries
!pip install -q langchain accelerate chromadb faiss-cpu sentence-transformers transformers langchain-huggingface langchain_experimental
!pip install opendatasets

## Data Loading & Preprocessing
* To test our model on critical e-commerce challenges, such as customer churn, demand forecasting, and cross-selling, we need to load and prepare a clean dataset.
* This step ensures the data is ready for analysis and model training.

In [ ]:
import opendatasets as od
od.download('https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce')

Skipping, found downloaded files in "./brazilian-ecommerce" (use force=True to force download)


In [ ]:
import pandas as pd

orders = pd.read_csv('/content/brazilian-ecommerce/olist_orders_dataset.csv')
order_items = pd.read_csv('/content/brazilian-ecommerce/olist_products_dataset.csv')
products = pd.read_csv('/content/brazilian-ecommerce/olist_order_items_dataset.csv')
order_payments = pd.read_csv('/content/brazilian-ecommerce/olist_order_payments_dataset.csv')
reviews = pd.read_csv('/content/brazilian-ecommerce/olist_order_reviews_dataset.csv')
sellers = pd.read_csv('/content/brazilian-ecommerce/olist_sellers_dataset.csv')
geolocation = pd.read_csv('/content/brazilian-ecommerce/olist_geolocation_dataset.csv')
customers = pd.read_csv('/content/brazilian-ecommerce/olist_customers_dataset.csv')
product_category = pd.read_csv('/content/brazilian-ecommerce/product_category_name_translation.csv')

## Some Explorition & Preprocessing

In [ ]:
merged_data = orders.merge(products, on='order_id', how='left')
merged_data = merged_data.merge(order_items[['product_id', 'product_category_name']], on='product_id', how='left')
merged_data = merged_data.merge(order_payments, on='order_id', how='left')
merged_data = merged_data.merge(reviews, on='order_id', how='left')
merged_data = merged_data.merge(sellers, on='seller_id', how='left')
merged_data = merged_data.merge(customers, on='customer_id', how='left')
merged_data = merged_data.merge(product_category, on='product_category_name', how='left')
display(merged_data.head().T)

,0,1,2,3,4
order_id,e481f51cbdc54678b7cc49136f2d6af7,e481f51cbdc54678b7cc49136f2d6af7,e481f51cbdc54678b7cc49136f2d6af7,53cdb2fc8bc7dce0b6741e2150273451,47770eb9100c2d0c44946d9cf07ec65d
customer_id,9ef432eb6251297304e76186b10a928d,9ef432eb6251297304e76186b10a928d,9ef432eb6251297304e76186b10a928d,b0830fb4747a6c6d20dea0b8c802d7ef,41ce2a54c0b03bf3443c3d931a367089
order_status,delivered,delivered,delivered,delivered,delivered
order_purchase_timestamp,2017-10-02 10:56:33,2017-10-02 10:56:33,2017-10-02 10:56:33,2018-07-24 20:41:37,2018-08-08 08:38:49
order_approved_at,2017-10-02 11:07:15,2017-10-02 11:07:15,2017-10-02 11:07:15,2018-07-26 03:24:27,2018-08-08 08:55:23
order_delivered_carrier_date,2017-10-04 19:55:00,2017-10-04 19:55:00,2017-10-04 19:55:00,2018-07-26 14:31:00,2018-08-08 13:50:00
order_delivered_customer_date,2017-10-10 21:25:13,2017-10-10 21:25:13,2017-10-10 21:25:13,2018-08-07 15:27:45,2018-08-17 18:06:29
order_estimated_delivery_date,2017-10-18 00:00:00,2017-10-18 00:00:00,2017-10-18 00:00:00,2018-08-13 00:00:00,2018-09-04 00:00:00
order_item_id,1.0,1.0,1.0,1.0,1.0
product_id,87285b34884572647811a353c7ac498a,87285b34884572647811a353c7ac498a,87285b34884572647811a353c7ac498a,595fac2a385ac33a80bd5114aec74eb8,aa4383b373c6aca5d8797843e5594415


In [ ]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119143 entries, 0 to 119142
Data columns (total 33 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       119143 non-null  object 
 1   customer_id                    119143 non-null  object 
 2   order_status                   119143 non-null  object 
 3   order_purchase_timestamp       119143 non-null  object 
 4   order_approved_at              118966 non-null  object 
 5   order_delivered_carrier_date   117057 non-null  object 
 6   order_delivered_customer_date  115722 non-null  object 
 7   order_estimated_delivery_date  119143 non-null  object 
 8   order_item_id                  118310 non-null  float64
 9   product_id                     118310 non-null  object 
 10  seller_id                      118310 non-null  object 
 11  shipping_limit_date            118310 non-null  object 
 12  price                         

In [ ]:
merged_data=merged_data.drop(columns=["order_id","customer_id","order_approved_at","order_delivered_carrier_date","order_item_id"
,"product_id","seller_id","shipping_limit_date","payment_sequential"
,"payment_installments","review_id","review_creation_date","review_answer_timestamp"
,"seller_zip_code_prefix","seller_state","seller_city"
,"customer_unique_id","customer_zip_code_prefix","review_comment_title","review_comment_message","order_purchase_timestamp"])

In [ ]:
merged_data.isna().sum()

,0
order_status,0
order_delivered_customer_date,3421
order_estimated_delivery_date,0
price,833
freight_value,833
product_category_name,2542
payment_type,3
payment_value,3
review_score,997
customer_city,0


In [ ]:
merged_data["product_category_name_english"] = merged_data["product_category_name_english"].fillna(merged_data["product_category_name_english"].mode()[0])
merged_data['payment_type'] = merged_data['payment_type'].fillna(merged_data['payment_type'].mode()[0])

# For numerical columns, fill with mean
for col in ['review_score', 'payment_value', 'freight_value', 'price','delivery_time_difference']:
     if col in merged_data.columns:
        merged_data[col] = merged_data[col].fillna(merged_data[col].mean())

merged_data.drop(columns=["order_estimated_delivery_date","order_delivered_customer_date","product_category_name"],inplace=True)

In [ ]:
merged_data.duplicated().sum()

np.int64(15850)

In [ ]:
merged_data.drop_duplicates(inplace=True)

In [ ]:
merged_data.sample()

,order_status,price,freight_value,payment_type,payment_value,review_score,customer_city,customer_state,product_category_name_english
55064,delivered,68.9,15.92,boleto,84.82,5.0,rio de janeiro,RJ,telephony


In [ ]:
merged_data = merged_data.rename(columns={
    'product_category_name_english': 'category',
    'review_score': 'review',
    'freight_value': 'freight',
    'payment_type': 'payment_method'
})

In [ ]:
data = merged_data

In [ ]:
data.sample(5)

,order_status,price,freight,payment_method,payment_value,review,customer_city,customer_state,category
90042,canceled,20.0,12.80,credit_card,32.80,1.0,sao jose do rio preto,SP,sports_leisure
85869,delivered,117.0,18.92,credit_card,135.92,5.0,sao joao do manteninha,MG,watches_gifts
87980,delivered,59.0,16.17,credit_card,75.17,4.0,venancio aires,RS,furniture_decor
6302,delivered,44.9,18.46,voucher,47.78,5.0,contagem,MG,sports_leisure
76323,delivered,64.9,14.21,credit_card,65.27,5.0,curitiba,PR,baby


## Building the Language Model (LLM)
- We will use the **google/flan-t5-xl** model as the core of our application.
- Since we'll be working with CSV data, we will specifically employ a structured retrieval RAG approach to optimize its performance.

### LLM For Analyze The Data

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain.chains import RetrievalQA

In [ ]:
# Language Model Setup
# This cell sets up the Large Language Model (LLM) and its pipeline.
# Load the tokenizer and model for the T5 family.
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-xl")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-xl")

# Create a text-generation pipeline using the model and tokenizer.
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=1024, device=0)

# Wrap the pipeline in a LangChain HuggingFacePipeline for easy integration.
llm = HuggingFacePipeline(pipeline=pipe)

print("HuggingFace pipeline and LLM initialized successfully.")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu


HuggingFace pipeline and LLM initialized successfully.


In [ ]:
# A dictionary mapping a keyword to the corresponding pandas expression.
PANDAS_EXPRESSIONS = {
    # General order information
    "total_orders": "len(data)",
    "order_statuses_count": "data['order_status'].value_counts()",
    "unique_product_categories_count": "data['category'].nunique()",

    # Financial insights
    "total_sales_value": "data['price'].sum()",
    "total_freight_value": "data['freight'].sum()",
    "total_payment_value": "data['payment_value'].sum()",
    "avg_payment_value": "data['payment_value'].mean()",
    "avg_freight_value": "data['freight'].mean()",
    "avg_price_per_product": "data['price'].mean()",
    "most_expensive_product_price": "data['price'].max()",
    "least_expensive_product_price": "data['price'].min()",
    "median_price": "data['price'].median()", # New: Median price
    "std_dev_price": "data['price'].std()", # New: Standard deviation of prices

    # Product and category insights
    "count_orders_per_category": "data.groupby('category')['order_status'].count()",
    "avg_price_per_category": "data.groupby('category')['price'].mean()",
    "top_3_categories_by_sales": "data.groupby('category')['price'].sum().nlargest(3)",
    "top_5_categories_by_sales": "data.groupby('category')['price'].sum().nlargest(5)",
    "bottom_5_categories_by_sales": "data.groupby('category')['price'].sum().nsmallest(5)",
    "most_popular_category": "data['category'].mode()[0]", # New: Most popular category

    # Payment and review analysis
    "reviews_per_category": "data.groupby('category')['review'].count()",
    "most_common_payment_type": "data['payment_method'].mode()[0]",
    "avg_review_score": "data['review'].mean()",
    "reviews_by_score": "data['review'].value_counts().sort_index()",
    "reviews_per_state": "data.groupby('customer_state')['review'].mean()", # New: Average review score per state

    # Geographic data
    "city_with_most_orders": "data['customer_city'].mode()[0]",
    "state_with_most_orders": "data['customer_state'].mode()[0]",
    "top_5_cities_by_orders": "data['customer_city'].value_counts().nlargest(5)",
    "top_5_states_by_orders": "data['customer_state'].value_counts().nlargest(5)",
    "top_3_states_by_sales": "data.groupby('customer_state')['price'].sum().nlargest(3)",
    "sales_per_state": "data.groupby('customer_state')['price'].sum()", # New: Total sales per state
    "orders_by_state_and_city": "data.groupby(['customer_state', 'customer_city'])['order_status'].count().sort_values(ascending=False)", # New: Orders by state and city
}

In [ ]:
def run_query_with_llm(query, df):
    """
    Asks the LLM to identify a keyword, executes the pandas expression,
    and then asks the LLM to format the result into a readable sentence.
    """
    # Step 1: Ask the LLM for a keyword.
    keyword_prompt = f"""
    You are an expert data analyst. You are given a pandas DataFrame named 'data'.
    Your task is to identify which of the following keywords best answers the user's query:

    Keywords: {list(PANDAS_EXPRESSIONS.keys())}

    Please provide only the single keyword that is the best match. Do not provide any other text or explanation.

    Query: {query}

    Response:
    """

    try:
        keyword = llm.invoke(keyword_prompt).strip()

        if keyword in PANDAS_EXPRESSIONS:
            expression_to_run = PANDAS_EXPRESSIONS[keyword]
            print(f"Executing this expression: {expression_to_run}")

            # Step 2: Execute the code to get the raw result.
            raw_result = eval(expression_to_run, {'data': df, 'pd': pd})

            # Step 3: Create a new prompt to format the output.
            formatting_prompt = f"""
            You are an expert data analyst. The user asked a question and you have the result of a data query.
            Please format the raw result into a clear, professional, and conversational sentence.
            Do not just print the numbers. Explain what they mean.

            User Query: {query}
            Raw Result: {raw_result}

            Formatted Answer:
            """

            # Step 4: Ask the LLM to format the result.
            formatted_answer = llm.invoke(formatting_prompt).strip()

            return formatted_answer

        else:
            return f"The LLM returned an invalid keyword: {keyword}"

    except Exception as e:
        return f"An error occurred while executing the code: {e}"

print("Custom query function defined with a new, more robust approach.")

# Cell 4: Interactive Query with a continuous loop
# This cell takes user input in a loop and prints the final answer.
while True:
    user_query = input("Give me Your Question About Your Data (type 'exit' to quit): ")
    if user_query.lower() == 'exit':
        break
    response = run_query_with_llm(user_query, data)
    print(f"Insight: {response}")

Custom query function defined with a new, more robust approach.
Give me Your Question About Your Data (type 'exit' to quit): exit


## LLM For Recommenditions

In [ ]:
def run_query_with_llm(query, df):
    """
    Asks the LLM to identify a keyword, executes the pandas expression,
    and then asks the LLM to format the result into a readable sentence.
    Returns both the formatted answer and the keyword.
    """
    keyword_prompt = f"""
    You are an expert data analyst. You are given a pandas DataFrame named 'data'.
    Your task is to identify which of the following keywords best answers the user's query:

    Keywords: {list(PANDAS_EXPRESSIONS.keys())}

    Please provide only the single keyword that is the best match. Do not provide any other text or explanation.

    Query: {query}

    Response:
    """
    try:
        keyword = llm.invoke(keyword_prompt).strip()
        if keyword in PANDAS_EXPRESSIONS:
            expression_to_run = PANDAS_EXPRESSIONS[keyword]
            print(f"Executing this expression: {expression_to_run}")
            raw_result = eval(expression_to_run, {'data': df, 'pd': pd})
            formatting_prompt = f"""
            You are an expert data analyst. The user asked a question and you have the result of a data query.
            Please format the raw result into a clear, professional, and conversational sentence.
            Do not just print the numbers. Explain what they mean.
            User Query: {query}
            Raw Result: {raw_result}
            Formatted Answer:
            """
            formatted_answer = llm.invoke(formatting_prompt).strip()
            return formatted_answer, keyword
        else:
            return f"The LLM returned an invalid keyword: {keyword}", None
    except Exception as e:
        # Fixed: This line now returns two values to prevent the ValueError
        return f"An error occurred while executing the code: {e}", None

print("Custom query function defined with a new, more robust approach.")

Custom query function defined with a new, more robust approach.


## We will Test The Model Now By Asking Some Qs About The Data

What is the average total price of an order?    

What is the total value of all payments?

Which payment method is the most common?

What are the top 5 categories by total sales value?

What is the average price for each product category?

How many unique product categories are there?

In [ ]:
# Interactive Query with a continuous loop for Insights and Recommendations
# This cell takes user input, generates an insight, and then generates an e-commerce recommendation.

# Create a second LLM instance, as requested.
llm2 = llm

# Define the prompt for generating e-commerce recommendations.
RECOMMENDATION_PROMPT_TEMPLATE = """
You are an expert e-commerce marketing consultant. Your task is to provide a detailed, actionable, and comprehensive recommendation to a business owner based on a data-driven insight.

Please use the following format for your response:
### Key Insight Summary
Briefly summarize the insight you've been given.

### Importance for the Business
Explain why this insight is relevant and important for an e-commerce business. What opportunities or challenges does it present?

### Actionable Recommendations
Provide a list of 2-3 specific and practical steps the business owner can take to leverage this insight. Be creative and think about marketing, inventory, or customer strategy.

### Expected Impact
Conclude with a sentence about the potential positive impact of these actions.

Insight from data analyst: {insight}

Detailed Recommendation:
"""

while True:
    user_query = input("Give me Your Question About Your Data (type 'exit' to quit): ")
    if user_query.lower() == 'exit':
        break

    # Step 1: Get the insightful answer from the data analyst.
    insightful_answer = run_query_with_llm(user_query, data)
    print(f"Insight: {insightful_answer}")

    # Step 2: Use the second LLM to generate a detailed recommendation.
    recommendation_prompt = RECOMMENDATION_PROMPT_TEMPLATE.format(insight=insightful_answer)
    recommendation = llm2.invoke(recommendation_prompt).strip()

    print(f"Recommendation: {recommendation}\n" + "-"*50)

Give me Your Question About Your Data (type 'exit' to quit): exit


- The Model Is Already Make Insights and Give critical Recommendations
- Now We Will Build A localy Strealit App

## Streamlit App For Analytics ChatBot
This section is for the final testing of the chatbot to ensure it functions as expected.

In [ ]:
# --- 1. Install necessary libraries
# This includes all the libraries needed for the app to run.
!pip install streamlit pyngrok transformers accelerate langchain_community --quiet
!pip install --upgrade "huggingface_hub[cli]"

  Using cached huggingface_hub-1.1.4-py3-none-any.whl.metadata (13 kB)
Using cached huggingface_hub-1.1.4-py3-none-any.whl (515 kB)
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.36.0
    Uninstalling huggingface-hub-0.36.0:
      Successfully uninstalled huggingface-hub-0.36.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.57.1 requires huggingface-hub<1.0,>=0.34.0, but you have huggingface-hub 1.1.4 which is incompatible.


-----------------------------------------------------------------
-----------------------------------------------------------------
-----------------------------------------------------------------

# Ecommerce Assistant
This is our Second Feature in The system
## Environment Setup & Model Loading
- We will use the **Mistral-7B-Instruct-v0.2** model
- Here we'll prepare all the necessary tools and libraries to build our application.

In [ ]:
# Install necessary libraries for RAG
!pip install -qU transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 11.6 MB/s eta 0:00:00


In [ ]:
# Import required modules
import os
import torch
from transformers import BitsAndBytesConfig, AutoTokenizer, AutoModelForCausalLM

One line: The LLM identifies a keyword from PANDAS_EXPRESSIONS, executes the corresponding Pandas expression on the full dataset, and formats the result into a human-readable sentence.

Two lines: Very precise and supports all types of queries, but slow on large datasets because each query requires the LLM twice.

In [ ]:
# --- app_gradio_brazilian_ecommerce_full_updated.py ---
import gradio as gr
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain_community.llms import HuggingFacePipeline

orders = pd.read_csv('/content/brazilian-ecommerce/olist_orders_dataset.csv')
order_items = pd.read_csv('/content/brazilian-ecommerce/olist_order_items_dataset.csv')
products = pd.read_csv('/content/brazilian-ecommerce/olist_products_dataset.csv')
order_payments = pd.read_csv('/content/brazilian-ecommerce/olist_order_payments_dataset.csv')
reviews = pd.read_csv('/content/brazilian-ecommerce/olist_order_reviews_dataset.csv')
customers = pd.read_csv('/content/brazilian-ecommerce/olist_customers_dataset.csv')
product_category = pd.read_csv('/content/brazilian-ecommerce/product_category_name_translation.csv')

# --- Merge datasets ---
products = products.merge(product_category, on='product_category_name', how='left')
order_items_products = order_items.merge(products, on='product_id', how='left')
data = orders.merge(order_items_products, on='order_id', how='left')
data = data.merge(customers, on='customer_id', how='left')
data = data.merge(reviews[['order_id', 'review_score']], on='order_id', how='left')
data = data.merge(order_payments[['order_id', 'payment_type']], on='order_id', how='left')

# --- Initialize LLM ---
llm_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
llm_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
device = 0 if torch.cuda.is_available() else -1
llm_pipe = pipeline("text2text-generation", model=llm_model, tokenizer=llm_tokenizer, max_length=512, device=device)
llm = HuggingFacePipeline(pipeline=llm_pipe)

# --- Pandas Expressions (Updated) ---
PANDAS_EXPRESSIONS = {
    "total_orders": "len(data)",
    "order_statuses_count": "data['order_status'].value_counts()",
    "unique_product_categories_count": "data['product_category_name_english'].nunique()",
    "total_sales_value": "data['price'].sum()",
    "avg_price_per_product": "data['price'].mean()",
    "avg_price_per_order": "data.groupby('order_id')['price'].sum().mean()",  # جديد
    "count_orders_per_category": "data.groupby('product_category_name_english')['order_id'].count()",
    "avg_price_per_category": "data.groupby('product_category_name_english')['price'].mean()",
    "top_5_categories_by_sales": "data.groupby('product_category_name_english')['price'].sum().nlargest(5)",
    "most_common_payment_type": "data['payment_type'].mode()[0]",
    "avg_review_score": "data['review_score'].mean()"
}

# --- Gradio function ---
def run_query_with_llm_gradio(query):
    if data.empty:
        return "Data is empty or failed to load."

    keyword_prompt = f"""
    You are an expert data analyst. You are given a pandas DataFrame named 'data'.
    Your task is to identify which of the following keywords best answers the user's query:

    Keywords: {list(PANDAS_EXPRESSIONS.keys())}

    Please provide only the single keyword that is the best match. Do not provide any other text or explanation.

    Query: {query}

    Response:
    """
    try:
        keyword = llm.invoke(keyword_prompt).strip()
        if keyword not in PANDAS_EXPRESSIONS:
            return f"The LLM returned an invalid keyword: {keyword}"

        expression_to_run = PANDAS_EXPRESSIONS[keyword]
        raw_result = eval(expression_to_run, {'data': data, 'pd': pd})

        formatting_prompt = f"""
        You are an expert data analyst. The user asked a question and you have the result of a data query.
        Please format the raw result into a clear, professional, and conversational sentence.
        Do not just print the numbers. Explain what they mean.

        User Query: {query}
        Raw Result: {raw_result}

        Formatted Answer:
        """
        formatted_answer = llm.invoke(formatting_prompt).strip()
        return formatted_answer
    except Exception as e:
        return f"An error occurred while processing your query: {e}"

# --- Gradio Interface ---
iface = gr.Interface(
    fn=run_query_with_llm_gradio,
    inputs=gr.Textbox(label="Ask a question about your data"),
    outputs="text",
    title="Brazilian E-commerce Analytics Chatbot",
    description="Ask questions about the Brazilian E-commerce dataset. The chatbot will provide insights based on the merged dataset."
)

if __name__ == "__main__":
    iface.launch(share=True)


Device set to use cpu


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://30955c088bf2fb96c9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


One line: Aggregates the top 1000 orders, converts each order into a text chunk, uses FAISS to retrieve the top 5 relevant chunks, and asks the LLM to generate an answer.

Two lines: Very fast, good for immediate responses, but less accurate if the question involves data outside the top 1000 orders.

In [ ]:
# --- app_gradio_rag_brazilian_ecommerce_fast1000_fixed.py ---
import gradio as gr
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# --- Load and merge datasets ---
orders = pd.read_csv('/content/brazilian-ecommerce/olist_orders_dataset.csv')
order_items = pd.read_csv('/content/brazilian-ecommerce/olist_order_items_dataset.csv')
products = pd.read_csv('/content/brazilian-ecommerce/olist_products_dataset.csv')
customers = pd.read_csv('/content/brazilian-ecommerce/olist_customers_dataset.csv')
product_category = pd.read_csv('/content/brazilian-ecommerce/product_category_name_translation.csv')

# Merge products with category translation
products = products.merge(product_category, on='product_category_name', how='left')
order_items_products = order_items.merge(products, on='product_id', how='left')
data = orders.merge(order_items_products, on='order_id', how='left')
data = data.merge(customers, on='customer_id', how='left')

# --- Aggregate per order and take top 1000 orders only ---
order_summary = data.groupby('order_id').agg({
    'customer_id': 'first',
    'product_category_name_english': lambda x: ', '.join([str(i) for i in x.dropna().unique()]),
    'price': 'sum',
    'order_status': 'first'
}).reset_index()

top_orders = order_summary.head(1000)

# --- Create text chunks for RAG ---
chunks = []
for _, row in top_orders.iterrows():
    text = f"Order {row['order_id']} by {row['customer_id']}: products {row['product_category_name_english']}, total price {row['price']}, status {row['order_status']}"
    chunks.append(text)

# --- Create vector store with embeddings ---
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_texts(chunks, embeddings)

# --- Initialize LLM ---
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
device = 0 if torch.cuda.is_available() else -1
llm_pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=512, device=device)

# --- Gradio function ---
def answer_with_rag(query):
    try:
        docs = vectorstore.similarity_search(query, k=5)
        context = "\n".join([doc.page_content for doc in docs])
        prompt = f"Use the following data to answer the question:\n\n{context}\n\nQuestion: {query}\nAnswer:"
        answer = llm_pipe(prompt)[0]['generated_text']
        return answer
    except Exception as e:
        return f"Error: {e}"

# --- Gradio Interface ---
iface = gr.Interface(
    fn=answer_with_rag,
    inputs=gr.Textbox(label="Ask a question about the Brazilian E-commerce data"),
    outputs="text",
    title="Brazilian E-commerce RAG Chatbot (Fast 1000 Orders)",
    description="Fast RAG chatbot using top 1000 orders only for quick responses."
)

if __name__ == "__main__":
    iface.launch(share=True)


Device set to use cpu


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e2891e576cab2ab906.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


What is the average total price of an order?    
What is the total value of all payments?


> Add blockquote


